In [1]:
import en_core_web_sm
import re
from os import listdir
import pandas as pd

# Getting all the names of the files

In [2]:
import os
import spacy 
arr = os.listdir()

pdf = []
docs = []

for file in arr:
    if file[-3:] == 'pdf':
        pdf.append(file)
    elif file[-4:] == 'docx':
        docs.append(file)
    

## PDF MINING

In [18]:
'''
Mining the pdf to get important info... 

'''
import textract
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
nlp = en_core_web_sm.load()



def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

In [4]:
for name in pdf: 
    text = convert(name)
    print('FILENAME: ',name)
    find_all(text)
    print("\n\n")

    

FILENAME:  CV-2020.pdf


NameError: name 'find_all' is not defined

In [ ]:
##TESTING
'''
['CV-2020.pdf',
 'Jule_Wirlmann_-_resume.pdf',
 'cv sydney pdf.pdf',
 'Rafael F Silva CV 2020.pdf',
 'Cv-csc.pdf',
 'Jeanine_Leow_CV.pdf',
 'RESUME_LioresSanJose.pdf']
'''

some_variable = convert("Cv-csc.pdf")
some_variable.split('\n')
clean_string = [i for i in some_variable.split('\n') if i != '']

for l in clean_string:
    print(l)


In [24]:
## Importing documents 


text = textract.process("Rabina Thapa cv.docx")
text = text.decode("utf-8")
text
clean_string = [i for i in text.split('\n') if i != '']
docs

['Subash Khadka CV  copy.docx',
 'XUAN LOC TRAN RESUME (1).docx',
 'Rabina Thapa cv.docx',
 'Resume20200000.. (1).docx',
 'CV FRANCESCA GIUSSANI  (1) copy.docx']

# Final codes 

In [33]:
# Getting the suburbs for the findint the location 

def get_subs(filename):
    data = pd.read_csv(filename)
    list_subs = []
    for ind,row in data.iterrows():
        list_subs.append(row["lol"])
        list_subs.append(row["lol1"])
        list_subs.append(row["lol2"])
        list_subs.append(row["lol3"])
    list_subs = list_subs[:-3]
    list_subs.append("Cranebrook")
    list_subs.remove('Sydney')
    return list_subs


# Removing the spaces from a string
def extract_number(line):
    line = line.replace("+", '')    
    line = line.replace(" ", '') 
    line = line.replace("Phone:", '')
    line = line .replace('(', '')
    line = line .replace(')', '')    
    lister = list(line)
    if line[0] == '6' and line[1] == '1':
        lister.pop(0)
        lister[0] = '0'
    elif line[1] == '6' and line[2] == '1':    ##Some pdfs have extra lines
        lister.pop(0)
        lister.pop(1)
        lister[0] = '0'
    line = "".join(lister)
    number_list = re.findall(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{5})', line)
    return number_list

# Get the name from the Pdf
def get_name(lines):
    lines = lines[:2]
    name = ''
    for l in lines:
        l = l.replace("\xa0", "")
        doc = nlp(l)
        for ent in doc.ents:
            if   ent.label_ == "PERSON":
                if ent.text not in ["Curriculum Vitae", 'Evens head']:   ## Add more words here if need be. 
                    name = ent.text 
                else:
                    pass
        if name != '':
            break
    
    if name == '':
        name = lines[0]
    return name

# Getting the email address
def get_email(lines):
    final_email = ''
    for l in lines: 
        if '@' in l:
            email_line = l
            break
    
    # This is the line containing the email, splitting as per space 
    email_line = email_line.split(" ")
    
    for word in email_line: 
        if '@' in word: 
            final_email = word
            break
    return final_email


# Getting the phone number
def get_phone(lines):
    phone_lines = []
    checking_num = ['+(61)','04', '+61', '61', '04' ]
    for l in lines:
        if any(ext in l for ext in checking_num):
            phone_lines.append(l)
    
    number = ''
    for i in phone_lines:
        num_list = extract_number(i)
        if len(num_list) != 0:
            number = num_list[0]
            break 
    return number
        

# Educational background, Returns a dict for different keys! Which can be used in the algorithm. 
def get_education(lines):
    educational_lines = []
    checking_edu = ['education', 'master', 'bachelor', 'academy', 'school', 'institute', 'acca', 'university', 'certificate']
    for l in lines:
        
        if any(ext in l.lower() for ext in checking_edu):
            educational_lines.append(l)
    
    dict_education = {}
    for line in educational_lines:
        if 'high' in line.lower() and 'school' in line.lower():
            dict_education['School'] = 'High School'
        
        elif 'school' in line.lower():
            dict_education['School'] = 'School'
        
        elif 'bachelor' in line.lower():
            dict_education['University_bachelor'] = line

        elif 'master' in line.lower():
            dict_education['University_master'] = line

        elif 'university' in line.lower():
            dict_education['University'] = line
        
        elif 'diploma' in line.lower():
            dict_education['Diploma'] = True
        else:
            dict_education['Other'] = line
            
        
    return dict_education

## Get the location from the whole string
def get_location(lines):
    filenames = listdir('suburbs/')
    list_of_files = [ filename for filename in filenames if filename.endswith( '.csv' )]
    list_of_subs = []
    final_sub = ''
    
    for file in list_of_files:
        list_of_subs += get_subs('suburbs/' + file) 
    
    for l in lines:
        for sub in list_of_subs:
            if sub.lower() in l.lower():
                final_sub =  sub
        if final_sub != '':
            break
    
    if final_sub == '':
        return 'Sydney'
    
    
    else:
        return final_sub
    
## Scraping the certificates held by the person
def get_certifications(lines):
    list_of_certificates = ['rsa', 'food', 'food safety training certified']
    
    certificates_held = []
    for l in lines:
        if 'cert' in l.lower() or 'license' in l.lower():
            if any(ext in l.lower() for ext in list_of_certificates):
                certificates_held.append(l)
                
    cert_dict = {}
    for l in certificates_held:
        if 'rsa' in l.lower():
            cert_dict['RSA'] = True
        elif 'food handling' in l.lower():
            cert_dict['Food Handling and Safety'] =True
        elif 'food safety training certified' in l.lower():
            cert_dict['Food Handling and Safety'] =True
    return cert_dict

## Scraping the experiences
def get_exp(lines):
    new_lines = []
    list_of_keywords = ['work experience', 'employment', 'work history', 'employment history', 'job experience', 'experiences', 'experience']
    for l in range(len(lines)):
        try:
            if any(ext in lines[l].lower() for ext in list_of_keywords):
                new_lines.append(lines[l:l+20])
        except:
            try:
                new_lines.append(lines[l:l+10])
                break
            except:
                break
    
    ## Joining the experience lines.
    exp_string = ''
    for i in new_lines:
        for words in i:
            exp_string += str(words)
        exp_string += ' '
    ## Nlp on the document..
    doc = nlp(exp_string)
    
    list_of_posts = ['airlines staff', 'barstaff', 'beverage staff', 'bar staff', 'chefs', 'cooks', 'guest services', 'gaming', 'housekeeping', 'kitchen hands', 'reservations', 'travel agents', 'waitress','waiter', 'sales assistant', 'cashier', 'barback', 'kitchen hand', 'runner', 'barista', 'cleaner']

    
    places_worked = []
    posts_held = []
    time_worked = []
    for ent in doc.ents:
        if ent.label_ == 'DATE':
            if '-' in ent.text or 'to' in ent.text:
                time_worked.append(ent.text)
        
        if any(ext in ent.text.lower() for ext in list_of_posts):
            posts_held.append(ent.text)
            
        elif ent.label_ == 'ORG' or 'hotel' in ent.text:
            if ent.text not in list_of_posts:
                places_worked.append(ent.text)
            
    return_dict = {"place_worked": [], "posts_held": [], "time_worked": []}
    
    
    for i in range(5):
        try:
            return_dict["time_worked"].append(time_worked[i])
            return_dict["posts_held"].append(posts_held[i])
            return_dict["place_worked"].append(places_worked[i])
            

        except IndexError:
            break
    
    ## Get the final posts for the usage further!!
    final_posts = []
    for i in lines:
        for j in list_of_posts:
            if j in i.lower():
                final_posts.append(j)
    
    
    print("HERE: ",exp_string)
    return return_dict, final_posts

def get_skills(lines):
    skill_list = ["hard-working", "hard working", "under pressure", "under-pressure", "eager to learn", "spreadsheets", "fast learner", "focused", "time management", "time-management", "communication skills", "attentiveness", "patience", "multitasking","multitasker", "cash management", "problem solving", "customer care", "polite", "punctual", "leadership", "cleaner", "cleaning"]
    final_sills = set()
    for l in lines:
        for skill in skill_list:
            if skill in l.lower():
                final_sills.add(skill)
    
    return list(final_sills)
            
                
                
def get_visa(lines):
    visa_list = ['subclass 462', 'subclass 417', 'subclass 408', 'subclass 500']
    visa_final = 'Not mentioned'
    for l in lines:
        for visa in visa_list:
            if visa in l.lower():
                visa_final = visa
    return visa_final 
    
def get_role(lines):
    role_list = ['part-time', 'part time', 'full time', 'full-time', 'casual role']
    role_final = 'Any role'
    for l in lines:
        for role in role_list:
            if role in l.lower():
                role_final = role
    return role_final   


def find_all(text):
    clean_string = [i for i in text.split('\n') if i != '']
    
    ## Get the name from the pf
    name = get_name(clean_string)
    print('NAME: ', name)
    
    ## Get the email 
    email = get_email(clean_string)
    print('email: ', email)
    
    ## Get the phone numbers
    number= get_phone(clean_string)
    print('number: ', number)
    
    ## Get the education
    education = get_education(clean_string)
    print("Education Dict: ", education )
    
    ## Get the location of the person. 
    location = get_location(clean_string)
    print("Location: ", location )
    
    ## Get the certifications held by the person
    certificates = get_certifications(clean_string)
    print("Certificates: ", certificates)
    
    ## Get the experiences of the person.
    exp, posts_held = get_exp(clean_string)
    print("Experiences: ", exp )
    print("Posts held: ", posts_held)
    
    ## Get the skills of the person.
    skills = get_skills(clean_string)
    print("Skills: ", skills)
    
    ## Get the Visa status of the person.
    visa = get_visa(clean_string)
    print("VISA: ", visa )
    
    ## Get the ROle type required by the person.
    role = get_role(clean_string)
    print("ROLE: ", role )

## Testing for final codes!

In [34]:
'''for name in pdf: 
    text = convert(name)
    print('FILENAME: ',name)
    find_all(text)
    print("\n\n")'''

    

'for name in pdf: \n    text = convert(name)\n    print(\'FILENAME: \',name)\n    find_all(text)\n    print("\n\n")'

In [35]:
for name in docs:
    text = textract.process(name)
    text = text.decode("utf-8")
    print('FILENAME: ',name)
    find_all(text)
    print("\n\n")
    

FILENAME:  Subash Khadka CV  copy.docx
NAME:  Subash Khadka
email:  subash.khadka007@gmail.com
number:  0451357123
Education Dict:  {'Other': 'Education', 'University_bachelor': 'Head Of Department, Bachelor in Hotel Management NATHM', 'School': 'School'}
Location:  Canterbury
Certificates:  {'Food Handling and Safety': True}
HERE:  ExperienceSeptember, 2014 – April 2016| Bartender Plus WaiterPark Hyatt Abu Dhabi | Abu Dhabi - UAEAdequate knowledge of wine, champagne, Cocktails, spirits, beer and other beverages.Taking order of food and beverage and serving. Knowledge of different kinds of service style.Able to encounter last minute changes.Lifting heavy weight as required in the operation.Leading team and regular casuals staffs.Supportive on providing customer with multiple choices.Reporting on-duty Supervisor and Assistant Manager.Adequate knowledge of Requisition.Proper knowledge of Inventory. Serving it right certification. Essential food safety training certified.Micros Handling a

Location:  Surry Hills
Certificates:  {}
HERE:  I am an experienced waitress with real pride and passion for what I do. I greet all of my customers with lots of energy and a BIG SMILE. I have the ability to build a rapport, even in a fast paced environment when the orders are flying in; and I love giving customers an awesome experience with great service delivery. ContactPHONE:0449970649EMAIL:francigiussani@outlook.comADDRESS:205/1 Randle St, Surry HillsTYPE OF VISA:Working Holiday VisaNATIONALITY:ItalianSkills:Mother tongue: Spanish, Italian Second language: English (B2) Good customer service skills	EDUCATIONGreenwich English CollegeNovember 2018 – February 2020 Hands-on experience with cash registers and ordering information systems (Revel POS & Toast POS)Good maths skills (including mental maths!)Attentiveness and patienceVery high standard of personal presentationGood organisational and multitasking skills, with proven ability to perform in a busy environmentActive listening and ef